In [ ]:
import matplotlib.pyplot as plt

from mxlpy import Model, Simulator, fns, plot

# SIR and SIRD models

In epidemiology, [compartmental models](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology) are often applied to model infectious diseases.  

Common compartments include ones for *S*usceptible, *I*nfectious and *R*ecovered individuals, which are included in the [SIR model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model).

In this model there are two transitions (`reactions` in mxlpy) between those compartments.

- susceptible individuals can become infected by contact with an infected person: $\beta S  I$
- infected people can recover with a rate proportional: $\gamma I$

These transitions are scaled by the average number of contacts per person per time ($\beta$) and the inverse of the average infection time $\gamma$.  


In [ ]:
def sir() -> Model:
    return (
        Model()
        .add_variables({"s": 0.9, "i": 0.1, "r": 0.0})
        .add_parameters({"beta": 0.2, "gamma": 0.1})
        .add_reaction(
            "infection",
            fns.mass_action_2s,
            args=["s", "i", "beta"],
            stoichiometry={"s": -1, "i": 1},
        )
        .add_reaction(
            "recovery",
            fns.mass_action_1s,
            args=["i", "gamma"],
            stoichiometry={"i": -1, "r": 1},
        )
    )


res = Simulator(sir()).simulate(100).get_result().unwrap_or_err()

fig, (ax1, ax2) = plot.two_axes(figsize=(7.5, 3.5))
_ = plot.lines(res.variables, ax=ax1)
_ = plot.lines(res.fluxes, ax=ax2)
ax1.set(xlabel="Time / a.u.", ylabel="Relative Population")
ax2.set(xlabel="Time / a.u.", ylabel="Rate of change")
plt.show()

We can now easily extend the original model by adding an additional compartment and transition.  

The [SIRD model]() for example differentiates between *r*ecovered and *d*eceased individuals.  

So there exists an additional compartment for *d*eceased individuals and a transition for *i*nfected to *d*eceased individuals, proportional to the amount of *i*nfected individuals and the mortality $\mu$ of the infection: $\mu I$

In [ ]:
def sird() -> Model:
    return (
        sir()
        .add_variable("d", 0.0)
        .add_parameter("mu", 0.01)
        .add_reaction(
            "death",
            fns.mass_action_1s,
            args=["i", "mu"],
            stoichiometry={"i": -1, "d": 1},
        )
    )


res = Simulator(sird()).simulate(100).get_result().unwrap_or_err()

fig, (ax1, ax2) = plot.two_axes(figsize=(7.5, 3.5))
_ = plot.lines(res.variables, ax=ax1)
_ = plot.lines(res.fluxes, ax=ax2)
ax1.set(xlabel="Time / a.u.", ylabel="Relative Population")
ax2.set(xlabel="Time / a.u.", ylabel="Rate of change")
plt.show()